In [1]:
!pip install flask_ngrok

In [2]:
!pip install pyngrok==4.1.1
!ngrok authtoken 2aM0IMgULySa4T6YyjlDpvLLdHT_495Ye7FhYqirifr3JDQkE

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=0d2a0b1a6883624d5560d9b0b125dc1e210e6b04f126db290d3e63d5bc3e9121
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from flask.templating import render_template
from flask import Flask
from flask_ngrok import run_with_ngrok
template_folder = 'gdrive/MyDrive/Colab Notebooks/App1/template'
app = Flask(__name__,template_folder=template_folder)
run_with_ngrok(app)
@app.route("/")
@app.route("/home")
def home():
  return render_template('home.html')

app.run()

In [ ]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Flask utils
from flask.templating import render_template
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
#from gevent.pywsgi import WSGIServer

from flask_ngrok import run_with_ngrok
template_folder = 'gdrive/MyDrive/Colab Notebooks/m-d/templates'
static_folder = 'gdrive/MyDrive/Colab Notebooks/m-d/static'
app = Flask(__name__,template_folder=template_folder,static_folder=static_folder)
run_with_ngrok(app)
# Define a flask app
#app = Flask(__name__)

# Model saved with Keras model.save()
MODEL_PATH ='gdrive/MyDrive/Colab Notebooks/m-d/plD_vgg19.h5'

# Load your trained model
model = load_model(MODEL_PATH)





def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)


    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x)
    preds = model.predict(x)
    preds = np.argmax(preds, axis=1)

    result = []
    for pred in preds:
        if pred == 0:
            result.append("Class 1: cocci")
        elif pred == 1:
            result.append("Class 2: healthy")
        elif pred == 2:
            result.append("Class 3: ncd")
        elif pred == 3:
            result.append("Class 4: salmo")

    return result

    # preds = model.predict(x)
    # preds=np.argmax(preds, axis=1)
    # if preds==0:
    #     preds="The Person is Infected With Pneumonia"
    # else:
    #     preds="The Person is not Infected With Pneumonia"


    # return preds



@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)
        result=preds
        return result
    return None


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e90d-34-83-23-20.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Jan/2024 12:51:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2024 12:51:36] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2024 12:51:36] "GET /static/js/main.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2024 12:51:37] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File 